# Radiant MLHub Data Access
Note: must enable the radiant earth mlhub python client. 
run pip install radiant-mlhub

In [1]:
from radiant_mlhub import Dataset, Collection, client, get_session
import tarfile
from pathlib import Path
import os
from io import BytesIO
from glob import glob
from tqdm.notebook import tqdm
import tifffile as tiff
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# MLHub API Key
os.environ['MLHUB_API_KEY'] = '673e0a44d16279e72d310dc580273c2cd52b37a5857321555e23e19a84af8eb4'

##  Data

In [3]:
dataset = Dataset.fetch('umd_mali_crop_type')

print(f'ID: {dataset.id}')
print(f'Title: {dataset.title}')
print('Collections:')
for collection in dataset.collections:
    print(f'* {collection.id}')

ID: umd_mali_crop_type
Title: 2019 Mali CropType Training Data
Collections:
* umd_mali_crop_type_labels
* umd_mali_crop_type_source


The two collections associated with this dataset are:

* umd_mali_crop_type_source: includes the multi-temporal bands of Sentinel-2
* umd_mali_crop_type_labels: includes the labels and field IDs


## Downloading Labels and Source Imagery

In [19]:
# output path where you want to download the data
output_path = Path("./data/").resolve()

archive_paths = dataset.download(output_dir=output_path)
for archive_path in archive_paths:
    print(f'Extracting {archive_path}...')
    with tarfile.open(archive_path) as tfile:
        tfile.extractall(path=output_path)

print('Done\n')

Extracting C:\Users\jenna\Documents\MCP\Spring_2022\MUSA650_RemoteSensing\MUSA650_Final_ChangEpstein\data\umd_mali_crop_type_labels.tar.gz...
Extracting C:\Users\jenna\Documents\MCP\Spring_2022\MUSA650_RemoteSensing\MUSA650_Final_ChangEpstein\data\umd_mali_crop_type_source.tar.gz...
Done



In [109]:
# The path to the source collection that was extracted from the download archive
collection_path = Path('./data/umd_mali_crop_type_source')

In [110]:
# check
collection_path

WindowsPath('data/umd_mali_crop_type_source')

In [115]:
# Available bands
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'CLD']

IGNORE BELOW

In [117]:
import os # to access the files on the local system

labels_path = Path("./data/umd_mali_crop_type_labels").resolve() # Label files
sources_path = Path("./data/umd_mali_crop_type_source").resolve() # source image files

In [82]:
labels_path

WindowsPath('C:/Users/jenna/Documents/MCP/Spring_2022/MUSA650_RemoteSensing/MUSA650_Final_ChangEpstein/data/umd_mali_crop_type_labels')

In [118]:
# get the path to all image source files
source_images = []
for root, dirs, files in os.walk(sources_path):
    for file in files:
        if(file.endswith("stac.json")):
            source_images.append(root)

In [119]:
dataset_df = pd.DataFrame(data = source_images,columns=["source_file"])

dataset_df.head()

,source_file
0,C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
1,C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
2,C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
3,C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
4,C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...


In [102]:
print(dataset_df)

                                            source_file
0     C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
1     C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
2     C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
3     C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
4     C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
...                                                 ...
1348  C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
1349  C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
1350  C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
1351  C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...
1352  C:\Users\jenna\Documents\MCP\Spring_2022\MUSA6...

[1353 rows x 1 columns]
